# 1. Scrapper 

This notebook summarizes the codes to webscrapping of 1.cronoscan and 2.Coingecko, and uploading it to Google Bigquery for Data storage. Using tools such as python request and google app script, we will extract the relavant data and load it into Bigquery. 

As a free tier of google cloud platform, this project is utilizes google app script, google cloud function, bigquery and public version of Tableau to provide ETL and delivery of Statistics on Cronos CEX and DEX. To manage Google Authentication, we'll be using pydata-google-auth instead of downloading service-accounts.

This notebook, also known as proto-scrapper, will be use to explore the data ingestion and summary of code to google app script. The main codes will be deployed to their respective servies.

Below contains the data that we wil be scrapping: 

1. Information about Cronos Centralization Exchange
- Scrap data from coingeko (API)
    - Top 10 Exchanges
    - Total Vol of all crypto trade
    - ..

2. Information about Cronos Decentralization Exchange
- Download file from cronosan - Charts & Statistics (Google App Script -> Google Sheet -> Upload to BQ)
    - The number of transaction of cronos chain
    - Cronos Chain Unique Addresses
    - Number of verified Contracts 

Link to Google App Script

    

# 1.1 Data Architecture

![image](images/data-scrapper.png)

Required python-packages
```
pip install requests
pip install google-cloud-bigquery
pip install pydata-google-auth
pip install pandas
```

In [2]:
from google.cloud import bigquery
import pydata_google_auth
import json
import pandas as pd
import requests 
import datetime
import hashlib
import io

In [6]:
# General functions
def bigquery_client(project_id):
    # This function connects to bigquery client
    credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/cloud-platform'],
    )

    # Use the credentials in other libraries, such as the Google BigQuery
    # client library.
    client = bigquery.Client(project=project_id, credentials=credentials)
    return client

def get_request(url, header):
    # This function perform get request from site and load the response as json
    page = requests.get(url)
    if page.status_code == 200:
        print("Successfully pulled data...")
        resp = json.loads(page.text)
        return resp
    else:
        print("Unsuccessful in pulling data...")
        return None
    
def cryto_com_general(result):
    # This function take in the request and put it into dataframe
    # Function extracts general info about cryto.com
    list_columns, list_values = [], []
    ingest_at = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    for row in result:
        if row != 'tickers':
            list_columns = list_columns + [row]
            list_values = list_values + [result[row]]
    
    list_columns = list_columns + ["ingested_at"]
    list_values = list_values + [ingest_at]
    df = pd.DataFrame(data=[list_values], columns=list_columns)
    return df


def get_crypto_com_tickers(result):
    # This function gets the tickers from crypto.com
    list_columns, list_values = [], []

    ingest_at = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Get the columns
    columns = [x for x in result['tickers'][0]]
    columns = columns + ['ingested_at']

    # Get the values
    for row in result['tickers']:
        values = []
        for x in row:
            values = values + [row[x]]
        values = values + [ingest_at]
        list_values = list_values + [values]

    df = pd.DataFrame(data=list_values, columns=columns)
    return df


def get_crypto_com_tickers_historical(result):
    # This function gets the tickers from crypto.com
    # Get the columns
    columns = ['timestamp', 'vol_btc', 'ingest_at']
    ingest_at = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    df = pd.DataFrame(columns=columns)

    # Get the values
    for row in result:
        df = df.append({"timestamp": row[0], "vol_btc":row[1], "ingest_at": ingest_at}, ignore_index=True)

    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df


def insert_into_bigquery(client, project_id, dataset_id, table_name, dataframe):
    # TODO(developer): Set table_id to the ID of the table to create.
    table_id = f"{project_id}.{dataset_id}.{table_name}"

    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        autodetect=True
    )

    job = client.load_table_from_dataframe(
        dataframe, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
    
    
def check_table_exist(client, table_id):
    # This function checks if home valuation csv exists
    # TODO(developer): Set table_id to the ID of the table to determine existence.
    # table_id = "your-project.your_dataset.your_table"
    try:
        client.get_table(table_id)  # Make an API request.
        print("Table {} already exists.".format(table_id))
        return True
    except:
        print("Table {} is not found.".format(table_id))
        return False
    
def main(url, table_name):
    header = "accept: application/json"
    project_id = "cryptoprojectcha"
    dataset_id = "raw_coingecko"
    client = bigquery_client(project_id)
    res = get_request(url, header)
    # Extract based on table_name
    if table_name == "raw_cronos_details":
        dataframe = get_crypto_com_tickers(res)
    elif table_name == "raw_cronos_tickers":
        dataframe = get_crypto_com_tickers_historical(res)
    else:
        dataframe = cryto_com_general(res)
    insert_into_bigquery(client, project_id, dataset_id, table_name, dataframe)

# 2. Coingecko Scrapper
## 2.1. Methodology
    - Use python Request module to request data
    - Extract and upload data into google bigquery
    - Use Google App script to trigger cron job (daily) to updating google sheet which will 
      be consumed by "Tableau"

In [7]:
data = {"https://api.coingecko.com/api/v3/exchanges/crypto_com": "raw_cronos_details", "https://api.coingecko.com/api/v3/exchanges/crypto_com/volume_chart?days=2": "raw_cronos_tickers",
       "https://api.coingecko.com/api/v3/exchanges/crypto_com": "raw_cronos_general"}

for key, value in data.items():
    url = key
    table_name = value
    main(url, table_name)

Successfully pulled data...


/Users/chakritthongek/Desktop/Cronos-Data-Analysis/venv/lib/python3.7/site-packages/google/cloud/bigquery/_pandas_helpers.py:472: UserWarning: Pyarrow could not determine the type of columns: status_updates.
  ", ".join(field.name for field in unknown_type_fields)


Loaded 1 rows and 23 columns to cryptoprojectcha.raw_coingecko.raw_cronos_general
Successfully pulled data...
Loaded 546 rows and 3 columns to cryptoprojectcha.raw_coingecko.raw_cronos_tickers


# 3. Cronoscan Scrapper

## 3.1. Methodology

- Use google app script to download file directly to Google Drive
- Trigger Google Cloud function (set up similar to webhook) to perform ETL and load data into Bigquery
- Use Google App Script to trigger cron to update a google sheet which will be consumed by Tableau